In [70]:
from os import getenv
from pymongo import MongoClient
from pandas import DataFrame, Series, concat
from dotenv import load_dotenv
from altair import Chart, X, Y, Color, Tooltip
import requests

In [71]:
filepath = '/Users/jasongersing/PycharmProjects/fastApiProject/.env'
load_dotenv(filepath)
url = getenv('MONGO_URL')
database = "thruTheGrapevine"
collection_GB = "GrapeBuyers"
collection_GS = "GrapeSellers"

In [72]:
db_GB = MongoClient(url)[database][collection_GB]
db_GB.count_documents({})

100

In [73]:
db_GS = MongoClient(url)[database][collection_GS]
db_GS.count_documents({})

20

In [74]:
grape_buyers = DataFrame(db_GB.find(projection={"_id": False, "grapes_seeking": True, "volume_seeking": True}))
grape_buyers = grape_buyers.explode(column=["grapes_seeking", "volume_seeking"])
grape_buyers.rename(columns={"grapes_seeking": "variety", "volume_seeking": "tons"}, inplace=True)
grape_buyers["side"] = "Buyer"
grape_buyers["tons"] = Series([int(num) for num in grape_buyers["tons"]])
grape_buyers = grape_buyers.groupby(["variety", "side"]).agg({"tons": "sum"}).reset_index()
grape_buyers

,variety,side,tons
0,Cabernet Sauvignon,Buyer,480
1,Chardonnay,Buyer,485
2,Granache,Buyer,170
3,Marsanne,Buyer,450
4,Merlot,Buyer,430
5,Mourvedre,Buyer,310
6,Nebbiolo,Buyer,320
7,Petit Verdot,Buyer,550
8,Pinot blanc,Buyer,485
9,Pinot gris,Buyer,645


In [75]:
grape_sellers = DataFrame(db_GS.find(projection={"_id": False, "grapes_selling": True, "volume_selling": True}))
grape_sellers = grape_sellers.explode(column=["grapes_selling", "volume_selling"])
grape_sellers.rename(columns={"grapes_selling": "variety", "volume_selling": "tons"}, inplace=True)
grape_sellers["side"] = "Seller"
grape_sellers["tons"] = Series([int(num) for num in grape_sellers["tons"]])
grape_sellers = grape_sellers.groupby(["variety", "side"]).agg({"tons": "sum"}).reset_index()
grape_sellers

,variety,side,tons
0,Cabernet Sauvignon,Seller,120
1,Chardonnay,Seller,70
2,Granache,Seller,30
3,Marsanne,Seller,55
4,Merlot,Seller,45
5,Mourvedre,Seller,55
6,Nebbiolo,Seller,75
7,Petit Verdot,Seller,5
8,Pinot blanc,Seller,70
9,Pinot gris,Seller,180


In [76]:
total = concat([grape_buyers, grape_sellers])
total

,variety,side,tons
0,Cabernet Sauvignon,Buyer,480
1,Chardonnay,Buyer,485
2,Granache,Buyer,170
3,Marsanne,Buyer,450
4,Merlot,Buyer,430
5,Mourvedre,Buyer,310
6,Nebbiolo,Buyer,320
7,Petit Verdot,Buyer,550
8,Pinot blanc,Buyer,485
9,Pinot gris,Buyer,645


In [77]:
def title_fix(string: str) -> str:
    return string.title().replace("_", " ")


def stacked_bar_chart(df: DataFrame, column_1: str, column_2: str, column_3: str) -> Chart:
    return Chart(
        df,
        title=f"{title_fix(column_1)} by {title_fix(column_2)} by {title_fix(column_3)}",
    ).mark_bar().encode(
        x=X(column_3, title=title_fix(column_3), sort="-y"),
        y=Y(column_1),
        color=Color(column_2, title=title_fix(column_2)),
        tooltip=Tooltip([column_2, column_3, column_1])
    ).properties(
        width=480,
        height=400,
        padding=24,
    ).configure(
        legend={"padding": 24},
        title={"fontSize": 20, "offset": 24},
        view={"stroke": "#FFF"},
    )

In [78]:
stacked_bar_chart(total, "tons", "side", "variety" )

alt.Chart(...)

In [79]:
chart = requests.get('http://127.0.0.1:8000/graph/df-grapes-by-side').json()
Chart.from_dict(chart)

alt.Chart(...)